<a href="https://colab.research.google.com/github/JWKKWJ123/google-colab/blob/main/Heatmaps_of_models_on_AD_CN_tasks_on_ADNI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nibabel as nib
import math
import pandas as pd
import numpy as np
import os
import cv2
import einops
import glob
import time
import einops
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy import ndimage
from scipy.ndimage import gaussian_filter


In [ ]:
def plot_selected_slices(struct_arr, num_slices=6, cmap='gray', vmin=None, vmax=None, overlay=None,
                         overlay_cmap='hot', overlay_vmin=None, overlay_vmax=None, _class=None, iteration=0):
    """
    Plot slices of a 3D image (and an overlay) along every axis, restricted to the middle 25%-75% region.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from scipy import ndimage
    from matplotlib.gridspec import GridSpec

    if vmin is None:
        vmin = struct_arr.min()
    if vmax is None:
        vmax = struct_arr.max()
    if overlay_vmin is None and overlay is not None:
        overlay_vmin = overlay.min()
    if overlay_vmax is None and overlay is not None:
        overlay_vmax = overlay.max()
    print(vmin, vmax, overlay_vmin, overlay_vmax)

    # Create the figure with GridSpec for precise control
    fig = plt.figure(figsize=(num_slices * 2, 6))
    gs = GridSpec(3, num_slices, figure=fig, wspace=0.05, hspace=0.02)  # Reduced hspace to minimize row spacing

    # Define slice indices for the middle 25%-75% range
    slice_ranges = []
    for axis in range(3):
        start = int(struct_arr.shape[axis] * 0.25)
        end = int(struct_arr.shape[axis] * 0.75)
        indices = np.linspace(start, end, num_slices, dtype=int)
        slice_ranges.append(indices)

    # Add the overlay colorbar (if overlay is provided)
    cbar_ax = None
    if overlay is not None:
        cbar_ax = fig.add_axes([0.92, 0.2, 0.015, 0.6])  # Adjusted colorbar position and size

    for axis, axis_label in zip([0, 1, 2], ['x', 'y', 'z']):
        for i in range(num_slices):
            ax = fig.add_subplot(gs[axis, i])  # Use GridSpec to position subplots
            i_slice = slice_ranges[axis][i]

            ax.axis('off')
            ax.imshow(ndimage.rotate(np.take(struct_arr, i_slice, axis=axis), 90),
                      vmin=vmin, vmax=vmax, cmap=cmap, interpolation=None)
            ax.text(0.03, 0.97, '{}={}'.format(axis_label, i_slice), color='white',
                    horizontalalignment='left', verticalalignment='top', transform=ax.transAxes)

            if overlay is not None:
                im = ax.imshow(ndimage.rotate(np.take(overlay, i_slice, axis=axis), 90),
                               cmap=overlay_cmap, vmin=overlay_vmin, vmax=overlay_vmax,
                               interpolation=None, alpha=0.7)

    # Add colorbar for the overlay
    if overlay is not None:
        fig.colorbar(im, cax=cbar_ax)

    # Save the plotted heatmap slice (optional; specify appropriate file path)
    plt.show()
    #plt.close(fig)






In [ ]:
#colors = ['black','blue','orange','red']
colors = ['black','black','blue','orange','gold','tomato','red']
#colors = ['red','tomato','blue','orange','gold','black','black']
#colors = ['blue','green','black','gold','red']

color_list = ListedColormap(colors)

mycmap = LinearSegmentedColormap.from_list('mycmap', colors)